# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from LAPD     
The [dataset](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4) is a continuously updated file in csv format. It contains City of Los Angeles arrest data from 2010 to present. It can be retrieved simply by making requests without parameters. For more complicated API call, you may need to add query parameters.

In [25]:
import logging
import pandas as pd
from sodapy import Socrata
import utils

# download data
CRIME_SOURCE = "data.lacity.org"
client0 = Socrata(CRIME_SOURCE, None)
results = client0.get("yru6-6re4",where="arst_date > '2019-12-10T00:00:00.000'",limit = 1000000) # wait to be confirmed
print(results[:2])
print('...')
print(results[-2:])

[{'rpt_id': '5819970', 'arst_date': '2019-12-11T00:00:00.000', 'time': '0550', 'area': '18', 'area_desc': 'Southeast', 'rd': '1822', 'age': '27', 'sex_cd': 'M', 'descent_cd': 'B', 'chrg_grp_cd': '08', 'grp_description': 'Other Assaults', 'arst_typ_cd': 'F', 'charge': '69PC', 'chrg_desc': 'OBSTRUCT/RESIST EXECUTIVE OFFICER', 'location': '10300 S  FIGUEROA                     ST', 'location_1': {'latitude': '33.9428', 'longitude': '-118.2826', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}}, {'rpt_id': '5820637', 'arst_date': '2019-12-11T00:00:00.000', 'time': '2100', 'area': '14', 'area_desc': 'Pacific', 'rd': '1415', 'age': '42', 'sex_cd': 'M', 'descent_cd': 'W', 'chrg_grp_cd': '22', 'grp_description': 'Driving Under Influence', 'arst_typ_cd': 'M', 'charge': '23152(A)VC', 'chrg_desc': 'DRUNK DRIVING ALCOHOL/DRUGS', 'location': 'LINCOLN', 'crsst': 'PALMS', 'location_1': {'latitude': '33.9972', 'longitude': '-118.4592', 'human_address': '{"address": "", "city": "

## 2. Raw Data to Documents/Dicts      
Datetime conversion is made and age column is changed to numeric data type. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [5]:
import logging
import pandas as pd
from sodapy import Socrata
import utils
CRIME_SOURCE = "data.lacity.org"
DOWNLOAD_PERIOD = 15         # second
logger = logging.Logger(__name__)
utils.setup_logger(logger, 'data.log')


def download_crime(url=CRIME_SOURCE, start_date = '2019-12-08T00:00:00.000'):
    """Returns records from `CRIME_SOURCE` that includes crime and arrestee information.
    """
    client = Socrata(url, None)
    results = client.get("yru6-6re4",where=f"arst_date >= '{start_date}'", limit = 10000) # wait to be confirmed
    return results

In [26]:
from datetime import datetime

df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
# df['hour'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)
#df['month'] = df['arst_date'].apply(lambda x: datetime.strptime(x, '%Y%m').time())
df['age'] = pd.to_numeric(df['age'])
df.head()

,rpt_id,arst_date,time,area,area_desc,rd,age,sex_cd,descent_cd,chrg_grp_cd,grp_description,arst_typ_cd,charge,chrg_desc,location,location_1,crsst
0,5819970,2019-12-11,0550,18,Southeast,1822,27,M,B,08,Other Assaults,F,69PC,OBSTRUCT/RESIST EXECUTIVE OFFICER,10300 S FIGUEROA ST,"{'latitude': '33.9428', 'longitude': '-118.282...",NaN
1,5820637,2019-12-11,2100,14,Pacific,1415,42,M,W,22,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,LINCOLN,"{'latitude': '33.9972', 'longitude': '-118.459...",PALMS
2,5820507,2019-12-11,1920,03,Southwest,0311,29,M,H,08,Other Assaults,M,243(E)(1)PC,BATT EX-SPOUSE/FIANCEE/PERSN W/DATING REL,5500 CARLIN ST,"{'latitude': '34.031', 'longitude': '-118.3632...",NaN
3,5820531,2019-12-11,2145,16,Foothill,1684,53,M,H,04,Aggravated Assault,F,245(A)(1)PC,"ADW, NOT FIREARM, W/GBI",11700 SATICOY ST,"{'latitude': '34.2166', 'longitude': '-118.391...",NaN
4,190921205,2019-12-11,1810,09,Van Nuys,0901,21,M,H,13,Prostitution/Allied,M,647(B)(2)PC,ENGAGE/SOLICIT SEX FOR MONEY/ETC,WYANDOTTE,"{'latitude': '34.203', 'longitude': '-118.4662...",SEPULVEDA


## 3. Upsert MongoDB         
To avoid duplicate data entries between each run, the de-duplication mechanism makes sure that only a new record with a different report_id (rpt_id) will be inserted to the database. 

In [30]:
import pymongo
client = pymongo.MongoClient()

In [33]:
#import time
db = client.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
count = 0
for record in df.to_dict('records'):
    #start = time.time()
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

rows=495, update=495, insert=0
